taken from : https://www.kaggle.com/schmitzi/cleaning-titanic-data-and-running-scikitlearn


## Load the data

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("titanic.csv", sep=",", header=0)

print(data.columns)

Index(['pclass', 'survived', 'name', 'sex', 'Age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


Data structure

In [2]:

print(data.columns)
print(data.dtypes)

Index(['pclass', 'survived', 'name', 'sex', 'Age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')
pclass         int64
survived       int64
name          object
sex           object
Age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object


## Check the first few entires

In [3]:
data.head()


,pclass,survived,name,sex,Age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


## Statistics

In [4]:
data.describe()


,pclass,survived,Age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881138,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413493,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [5]:
data.corr()


,pclass,survived,Age,sibsp,parch,fare,body
pclass,1.000000,-0.312469,-0.408106,0.060832,0.018322,-0.558629,-0.034642
survived,-0.312469,1.000000,-0.055512,-0.027825,0.082660,0.244265,NaN
Age,-0.408106,-0.055512,1.000000,-0.243699,-0.150917,0.178740,0.058809
sibsp,0.060832,-0.027825,-0.243699,1.000000,0.373587,0.160238,-0.099961
parch,0.018322,0.082660,-0.150917,0.373587,1.000000,0.221539,0.051099
fare,-0.558629,0.244265,0.178740,0.160238,0.221539,1.000000,-0.043110
body,-0.034642,NaN,0.058809,-0.099961,0.051099,-0.043110,1.000000


## Estimate incomplete data

The data incomplete. It needs to be filled out. This is done by taking in consideration the the mean of the values and the mode in the case for the emmabrked.

In [6]:
data.dropna( axis=0, how="any", thresh=None, subset=['name'], inplace=True)


In [7]:

data['Age'].fillna(value=data['Age'].mean(), inplace=True)
data['fare'].fillna(value=data['fare'].mean(), inplace=True)
data['embarked'].fillna(value=(data['embarked'].value_counts().idxmax()), inplace=True)

In [8]:
## Extract titles and add a column.
# A title is not found it is defaulted as Mr.
def extract_title(row):
    if (isinstance(row['name'],np.int64)):
        print(row['name'])
        print('is considered as an int 64' )
        return 'Mr.'
    return row['name'].split(",")[1].split(".")[0]

data['Title'] = data.apply(extract_title, axis = 1)
#titles = pd.DataFrame(data.apply(lambda x: x.name.split(",")[1].split(".")[0], axis=1), columns=["Title"])

## Drop data that is not helpful

In [9]:
#Drop data that is not helpful.

data.drop('name', axis=1, inplace=True)
data.drop('cabin', axis=1, inplace=True)
data.drop('ticket', axis=1, inplace=True)
data.drop('body', axis=1, inplace=True)
# boats must go.


## Change the data into numbers

In [10]:
for col in data.select_dtypes(exclude=["number"]).columns:
    print("Converting column "+col+"...")
    data[col] = data[col].astype('category')
    print(data[col].cat.categories)
    data[col] = data[col].cat.codes


Converting column sex...
Index(['female', 'male'], dtype='object')
Converting column embarked...
Index(['C', 'Q', 'S'], dtype='object')
Converting column boat...
Index(['1', '10', '11', '12', '13', '13 15', '13 15 B', '14', '15', '15 16',
       '16', '2', '3', '4', '5', '5 7', '5 9', '6', '7', '8', '8 10', '9', 'A',
       'B', 'C', 'C D', 'D'],
      dtype='object')
Converting column home.dest...
Index(['?Havana, Cuba', 'Aberdeen / Portland, OR', 'Albany, NY',
       'Altdorf, Switzerland', 'Amenia, ND', 'Antwerp, Belgium / Stanton, OH',
       'Argentina', 'Asarum, Sweden Brooklyn, NY',
       'Ascot, Berkshire / Rochester, NY', 'Auburn, NY',
       ...
       'Wimbledon Park, London / Hayling Island, Hants',
       'Windsor, England New York, NY', 'Winnipeg, MB', 'Winnipeg, MN',
       'Woodford County, KY', 'Worcester, England', 'Worcester, MA',
       'Yoevil, England / Cottage Grove, OR', 'Youngstown, OH',
       'Zurich, Switzerland'],
      dtype='object', length=369)
Converti

In [11]:
data.head()

,pclass,survived,sex,Age,sibsp,parch,fare,embarked,boat,home.dest,Title
0,1,1,0,29.00,0,0,211.3375,2,11,308,9
1,1,1,1,0.92,1,2,151.5500,2,2,230,8
2,1,0,0,2.00,1,2,151.5500,2,-1,230,9
3,1,0,1,30.00,1,2,151.5500,2,-1,230,12
4,1,0,0,25.00,1,2,151.5500,2,-1,230,13


## Modifiy the table

* Place survived at the end of the table. -> https://cmdlinetips.com/2020/03/move-a-column-to-first-position-in-pandas-dataframe/

In [12]:
survived_col = data.pop('survived')
data.insert(len(data.columns), 'survived', survived_col)


## Save the data

In [13]:
data.to_csv("titanic_numerical_clean.csv", index = False, header = False)
data.to_csv("titanic_numerical_clean._With_Headers.csv", index = True)